# ETL Australian user items

Este notebook presenta la extracción, transformación y carga de el data frame de objetos de usuarios Australianos de Steam, junto con los criterios de por que se modifico, conservó o eliminó cada parte de la información.

---

# 1. Importamos las librerías necesarias.

In [1]:
import pandas as pd

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_1928\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 2. Cargamos los datos.

Usamos la función creada previamente ( consultar el notebook de Reseñas para más información ) para cargar los datos, debido a que este set de datos también presentaba problemas al cargarlo con pd.read_csv( ) por la estructura de este.

In [2]:
def json_problematico(path):
    '''
    Crea un dataframe cuando el método pd.read_json() falla debido a que el JSON origen tiene problemas de sintaxis como usar comillas distintas entre otras.

    requiere:
    pandas y ast.
    recibe:
    Ruta del archivo JSON.
    retorna:
    dataframe de pandas con dicho archivo JSON.
    '''
    # Importamos las librerías necesarias.
    import ast
    import pandas as pd
    # Una lista para guardar los resultados y luego convertirla a DataFrame.
    datos = []
    # Abrimos el archivo
    with open(path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                # Convertimos cada linea a un diccionario de python.
                json_line = ast.literal_eval(line)
                # Agregamos la linea a los datos.
                datos.append(json_line)
            # Tratamos los errores con un output en la linea que se produjo
            except:
                print(f"Error en la línea: {i+1}")
                continue
    # Creamos el data frame con los datos resultantes y lo retornamos.
    df = pd.DataFrame(datos)
    return df

In [3]:
df_inv = json_problematico('../Data/australian_users_items.json')

# 3. Primera vista y limpieza.

In [4]:
df_inv.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Nuevamente los items están en una lista de diccionarios y tendremos que realizar el mismo proceso que realizamos con el dataframe de reviews, por lo que empezamos eliminando la columna url que no usaremos, y limpiando el dataframe de valores nulos o duplicados que puedan afectar al script que nos ayude a descomprimir la columna items.

No usaré la columna steam_id ya que el identificador que se usa en otra base de datos es el user_id.

In [5]:
df_inv = df_inv.drop(columns=['user_url', 'steam_id']).reset_index(drop=True) # Eliminamos la columna no necesaria.

In [6]:
df_inv.isna().sum() # Verificamos los valores faltantes.

user_id        0
items_count    0
items          0
dtype: int64

Al no haber valores faltantes, seguimos con los duplicados.

In [7]:
df_inv[df_inv.duplicated(subset=['user_id', 'items_count'])] # Verificamos los valores duplicados.

,user_id,items_count,items
865,bokkkbokkk,0,[]
1732,Nikiad,109,"[{'item_id': '20', 'item_name': 'Team Fortress..."
2343,76561198079743094,48,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
2344,ImSeriouss,50,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
2394,76561198069124937,0,[]
...,...,...,...
48944,76561198035336388,5,"[{'item_id': '22600', 'item_name': 'Worms Relo..."
67355,76561198071790027,0,[]
69344,darkus0haos,122,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
73424,76561198080057659,39,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."


In [8]:
df_inv = df_inv.drop_duplicates(subset=['user_id', 'items_count']).reset_index(drop=True)

# 4. Transformación.

Ahora descomprimimos los valores dentro de la columna items.

No usaré las columnas item_name (ya que con el id identificaremos el juego dentro de la base de datos de juegos), ni playtime_2weeks ya que no la considero relevante y en su mayoría será 0, además la base de datos no es actual y esas 2 semanas no son en realidad las últimas 2 semanas, así que no sabemos si ese usuario en realidad jugó recientemente a algo.

In [9]:
import pandas as pd

keys = ['item_id', 'playtime_forever']  # Las columnas que queremos conservar de items.
# Almacenamos los resultados en una lista primero, ya que este dataframe es mucho más pesado.
data = []
# Iteramos sobre cada fila del df.
for i, items in enumerate(df_inv['items']):
    id = df_inv['user_id'].iloc[i]  # Extraemos el 'user_id'
    # Iteramos sobre los diccionarios en 'items' para extraer las claves específicas
    for item in items:
        # Creamos un diccionario con las claves y el user_id previamente extraido.
        row_data = {'id': id}
        for key in keys:
            row_data[key] = item.get(key, None)  # Si la clave no está presente, asignamos None
        # Asignamos el dato a la lista.
        data.append(row_data)
# Creamos el nuevo DataFrame.
df = pd.DataFrame.from_records(data)

Este script, no toma en cuenta los usuarios sin items, lo cual está bien, ya que tanto para el algoritmo de recomendación en base a un usuario, como para la API, necesitamos que el usuario tenga al menos un item.

In [10]:
del df_inv

---

# 5. Limpieza 2 ( Eliminación de juegos desconocidos ).

Recordemos que en el dataframe de juegos, hubo muchos registros que no existían, muchos de esos pueden ser juegos que tengan los usuarios en su inventario, pero que al buscarlos en la API para hacer procesos, no estarán y no solo será innecesaria la búsqueda, sino que también puede ocasionar errores, por lo que borraremos los juegos de los que no se tenga información en la base de datos de juegos.

In [11]:
df_games = pd.read_csv('../Data/Processed/games_api.csv')
df_games.head()

,genres,name,price,id,developer,year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018.0
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,0.00,643980,Secret Level SRL,2018.0
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017.0
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,0.99,767400,彼岸领域,2017.0
4,"['Action', 'Indie', 'Casual', 'Sports']",Log Challenge,2.99,773570,NaN,NaN


In [12]:
df_games['id'] = df_games['id'].astype(str) # Convertimos el id a string para que no de problemas con la comprobación.

In [13]:
df_filtered = df[df['item_id'].isin(df_games['id'])] # Creamos un df únicamente con los registros que haya en nuestro df de juegos.

In [14]:
# Verificamos qué tantos datos se pierden.
print(df.shape[0])
str(((df.shape[0] - df_filtered.shape[0] ) / df.shape[0]) * 100)[:4] + ' %' 

5094082


'16.6 %'

Vemos que de 5'094,082 datos, perdemos el 16.6 % ya que son registros de los cuales no tenemos información, por lo que no podremos hacer el EDA ni las funciones de la API correctamente con ellos.

In [15]:
df = df_filtered # Reemplazo el dataframe para continuar con el ETL.
del df_filtered

In [16]:
df.head()

,id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4244028 entries, 0 to 5094080
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   id                object
 1   item_id           object
 2   playtime_forever  int64 
dtypes: int64(1), object(2)
memory usage: 129.5+ MB


# 6 Carga.

Para terminar, para el API nos sirve cada registro tal cual como está, pero, para el EDA, no nos sirven los registros que tengan 0 horas en su tiempo de juego, esto debido a que la influencia del producto será muy baja o nula al solo haberlo comprado, mientras que en la API, el tener el producto implica que el usuario gastó dinero en él y una de las funciones nos pide esa información, por lo que guardaremos el dataframe de la API, eliminaremos los registros con 0 horas y los guardaremos para el EDA.

In [18]:
df.to_csv('../Data/Processed/items_api.csv', index=False)

In [19]:
df = df[df['playtime_forever'] > 0]

In [20]:
df.to_csv('../Data/Processed/items_recommend.csv', index=False)